In [ ]:
pip install patool


     |████████████████████████████████| 81kB 7.2MB/s 


In [ ]:
from google.colab import drive
!gdown --id 1tVOkgCXL5z_4pNaCfUL6qQHniIoliPWg

Downloading...
From: https://drive.google.com/uc?id=1tVOkgCXL5z_4pNaCfUL6qQHniIoliPWg
To: /content/car classification.rar
1.24GB [00:07, 158MB/s]


In [ ]:
from pyunpack import Archive
Archive('car classification.rar').extractall('./')

In [ ]:
from keras.layers import *
from keras import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil

In [ ]:
'''list_of_cars=os.listdir("generated_img/")
print("Training Data")
for f in list_of_cars:
    path = "generated_img/"+f
    print(f+ " "+str(len(os.listdir(path))))
print("\n Validation Data") 
for f in list_of_cars:
    path = "valimg/"+f
    print(f+ " "+str(len(os.listdir(path))))'''

'list_of_cars=os.listdir("generated_img/")\nprint("Training Data")\nfor f in list_of_cars:\n    path = "generated_img/"+f\n    print(f+ " "+str(len(os.listdir(path))))\nprint("\n Validation Data") \nfor f in list_of_cars:\n    path = "valimg/"+f\n    print(f+ " "+str(len(os.listdir(path))))'

In [ ]:
val_gen = ImageDataGenerator(rescale=1.0/255)
val_generator = val_gen.flow_from_directory(
    "valimg/",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)


Found 25249 images belonging to 196 classes.


In [ ]:
train_gen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_gen.flow_from_directory(
    "generated_img/",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 100618 images belonging to 196 classes.


In [ ]:
for x,y in train_generator:
    print(x.shape)
    print(y.shape)
    break

(32, 224, 224, 3)
(32, 196)


In [ ]:
from keras.applications.inception_v3 import InceptionV3

In [ ]:
model=InceptionV3(include_top=False,weights='imagenet',input_shape=(224,224,3))

87916544/87910968 [==============================] - 7s 0us/step


In [ ]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
av1=GlobalAveragePooling2D()(model.output)
fc1=Dense(1024,activation='relu')(av1)
fc2=Dense(512,activation='relu')(fc1)
d1=Dropout(0.5)(fc2)
fc2=Dense(196,activation='softmax')(d1)

In [ ]:
model_new=Model(inputs=model.input,outputs=fc2)

In [ ]:
for ix in range(len(model_new.layers)):
    print(ix,model_new.layers[ix])

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f75cdd41240>
1 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f757bc5feb8>
2 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f757bc071d0>
3 <tensorflow.python.keras.layers.core.Activation object at 0x7f757bc078d0>
4 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f757bc07668>
5 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f7570043198>
6 <tensorflow.python.keras.layers.core.Activation object at 0x7f75700433c8>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f7570043400>
8 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f752217c908>
9 <tensorflow.python.keras.layers.core.Activation object at 0x7f752217ccc0>
10 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f752217ccf8>
11 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f752219

In [ ]:
for ix in range(289):
    model_new.layers[ix].trainable=False

In [ ]:
from keras.optimizers import Adam

In [ ]:
adam=Adam(learning_rate=0.00003)

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1)

In [ ]:
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [ ]:
history = model_new.fit(
    train_generator,
    epochs = 25,
    steps_per_epoch = 3145,
    validation_data = val_generator,
    validation_steps=789,
    callbacks=[es]  
)

Epoch 1/25
3145/3145 [==============================] - 386s 123ms/step - loss: 4.5831 - accuracy: 0.0690 - val_loss: 3.6610 - val_accuracy: 0.2273
Epoch 2/25
3145/3145 [==============================] - 382s 121ms/step - loss: 3.5778 - accuracy: 0.1937 - val_loss: 2.8681 - val_accuracy: 0.3750
Epoch 3/25
3145/3145 [==============================] - 381s 121ms/step - loss: 2.9583 - accuracy: 0.2979 - val_loss: 2.3731 - val_accuracy: 0.4655
Epoch 4/25
3145/3145 [==============================] - 384s 122ms/step - loss: 2.5006 - accuracy: 0.3856 - val_loss: 2.0183 - val_accuracy: 0.5427
Epoch 5/25
3145/3145 [==============================] - 385s 122ms/step - loss: 2.1316 - accuracy: 0.4638 - val_loss: 1.7342 - val_accuracy: 0.5985
Epoch 6/25
3145/3145 [==============================] - 385s 122ms/step - loss: 1.8355 - accuracy: 0.5313 - val_loss: 1.5206 - val_accuracy: 0.6395
Epoch 7/25
3145/3145 [==============================] - 385s 122ms/step - loss: 1.5878 - accuracy: 0.5866 - val_

In [ ]:
model_new.save('model.h5')